# Chapter 3 - When Objects are alike

Duplicated code is considered evil. Steps to avoid duplicate code should be taken wherever possible.

There are many ways to merge code or objects that function similarly.

This chapter will focus on inheritance.

Specifically we will cover:

* Basic inheritance
* Inheriting from built-ins
* Multiple inheritance
* Polymorphism and duck typing

## Basic Inheritance

Technically every class we create uses inheritance.

All python classes are subclasses of the special class named _object_.

By default all classes inherit the object class.



In [1]:
class MySubClass(object):
    pass

# IS exactly the same as

class MySubClass():
    pass

A class being inherited from is called a superclass or parent class.

In the previous example the superclass is _object_. Even when not explicitly called.

To invoke inheritance simply include the name of the class in the paranthesis of the class definition.

In practice inheritance is used to provide additional functionality to an existing class.

**Example**

A contact manager that tracks the names and emails of people

In [2]:
class Contact:
    all_contacts = []
    
    def __init__(self, name, email):
        self.name = name
        self.email = email
        Contact.all_contacts.append(self)

The _all_contacts_ list is shared by all instances of this class. Because it is part of class definition.

The following can be used to access the all_contacts object

```python
Contact.all_contacts

# alternatively

self.all_contacts # will work on any object instantiated from Contact
```

If a field is not found on the object then python will look in the class definition for it.

> Warning
>
> Using self.all_contacts in the class definition will create a **new** instance variable. This will then only be acessible on that particular object.

Lets make a supplier class that inherits from the Contact class. We would like to add an order method, however it doesn't make sense to be able to order from everyone in the address book, it would only apply to suppliers.

In [3]:
class Supplier(Contact):
    def order (self, order):
        print("If this were a real system we would send "
             "'{}' order to '{}'".format(order, self.name))

In [4]:
c = Contact("Some Body", "somebody@example.net")
s = Supplier("Sup Plier", "supplier@example.net")

print(c.name, c.email, s.name, s.email)

Some Body somebody@example.net Sup Plier supplier@example.net


In [5]:
c.all_contacts

[<__main__.Contact at 0x1d9a4c26ba8>, <__main__.Supplier at 0x1d9a4c26b70>]

In [6]:
c.order("I need pliers")

AttributeError: 'Contact' object has no attribute 'order'

In [7]:
s.order("I need pliers")

If this were a real system we would send 'I need pliers' order to 'Sup Plier'


The Supplier class can do everything the Contact class can do.

And the extra thing which is handling orders.

## Extending built-ins

We can add functionality to built in classes.

In the contact class there is a list of all contacts defined. This could do with being searchable.

Let's create a contact list class which inherits from the list built in class and add a search method to it.

In [2]:
class ContactList(list):
    def search(self, name):
        '''Return all contacts that contain the search value 
        in their name.'''
        matching_contacts = []
        for contact in self:
            if name in contact.name:
                matching_contacts.append(contact)
        return matching_contacts
    
class Contact:
    all_contacts = ContactList()
    
    def __init__(self,name, email):
        self.name = name
        self.email = email
        self.all_contacts.append(self)

all_contacts is now an instance of the ContactList class.

The ContactList class extends the built in list class by adding a method that searches for names.

let's test

In [3]:
c1 = Contact("John A", "johna@example.net")
c2 = Contact("John B", "johnb@example.net")
c3 = Contact("Jenna C", "jennac@example.net")

In [4]:
[c.name for c in Contact.all_contacts.search('John')]

['John A', 'John B']

In [5]:
[] == list()

True

The \[ \] syntax calls the list() constructor under the hood.

\[ \] is refered to as syntax sugar.

the list data type is a class we can extend. The list itself extends the object class.

In [6]:
isinstance([], object)

True

Let's extend the dictionary class

In [7]:
class LongNameDict(dict):
    def longest_key(self):
        longest = None
        for key in self:
            if not longest or len(key) > len(longest):
                longest = key
        return longest

In [8]:
longkeys = LongNameDict()
longkeys['hello'] = 1
longkeys['longest yet'] = 5
longkeys['hello2'] = 'world'


In [9]:
longkeys.longest_key()

'longest yet'

Most built in types can be extended.

Built in classes that are commonly extended include:
* object
* list
* dict
* set
* file
* str

## Overriding and super

Inheritance is good for adding new behaviour.

What if we want to change existing behaviour? For example if we want to add a phone number for our close contacts.

The below shows how to override the init method in the subclass.

Overriding means replacing or altering a method of the superclass with a new method (with the same name).

When the method is called the definition for the subclass is used instead of the one in the superclass.

In [10]:
class Friend(Contact):
    def __init__(self, name, email, phone):
        self.name = name
        self.email = email
        self.phone = phone

Any method can be overidden.

The above example may cause some problems however:
* duplicate code to set up name and email.
* The friend class is not adding itself to all_contacts list we created in the contacts class

We need to execute the original \_\_init__ method that is part of the Contact class.

The 'super' function can do this for us. This returns the object as an instance of the parent class so it is possible to call the parent method directly.

In [11]:
class Friend(Contact):
    def __init__(self, name, email, phone):
        super.__init__(name, email)
        self.phone = phone

A 'super()' call can be made inside any method.

A 'super()' call can be made at any point in the method. Eg after input validation.

## Multiple inheritance

This refers to a subclass that inherits from more than one parent class.

This is contraversial, many experts recommend against this.

The most useful form of multiple inheritance is called a __mixin__. A mixin is meant to be inherited by some other class to provide extra functionality.

For example a 

In [12]:
class MailSender:
    def send_mail(self, message):
        print("Sending mail to " + self.email)
        # Add e-mail logic here